# Uppgift 1: Theodors område

In [5]:
import pandas as pd
import numpy as np
import plotly.express as px
from pathlib import Path

print("Reading file...")
olympics_csv = next(Path.cwd().parent.rglob("olympics_clean.csv"), None)

if olympics_csv:
    olympics = pd.read_csv(olympics_csv).drop('Name_hash', axis=1)
    print("file found.")
else:
    print("file not found.")

Reading file...
file found.


## Medaljer över tid

### Sommar:

In [14]:
norway = olympics.loc[olympics['NOC'] == 'NOR']
norway['Medal'].notna().sum()

np.int64(1033)

Medaljer över tid: